In [1]:
import os, glob, re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv("HOME") + "/aiffel/lyricist/data/lyrics/*"
txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, 'r') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터크기 : ", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터크기 :  0
Examples:
 []


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
    if sentence[-1] == ':': continue
        
    if idx > 9: break
    
    print(sentence)

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r"\1", sentence)
    sentence = re.sub(r'[""]+', "", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+"," ",sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

print(preprocess_sentence("This @_is ;;; sample   setnence"))

<start> this is sample setnence <end>


In [5]:
#len(raw_corpus[1].split())

In [7]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence.split()) >= 13: continue
    if sentence[-1] == ":": continue
    #if sentence == "[Hook]": continue
  
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:10]

[]

In [8]:
# count = 0
# for idx, sent in enumerate(raw_corpus):
#     if len(sent.split()) >= 13:
#         #print(idx,':',sent)
#         #print(idx, ":", len(sent.split()))
#         count += 1
#         if count > 50: break
# print(count)

In [9]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=12000,
    filters='',
    oov_token='<unk>'
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor.shape,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


ValueError: zero-size array to reduction operation maximum which has no identity

In [10]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break
    

NameError: name 'tokenizer' is not defined

In [11]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]

tgt_input = tensor[:, 1:]

#print(src_input[0])
#print(tgt_input[0])

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input, 
                                                          test_size=0.2,
                                                          random_state=42
                                                         )
#print(enc_train[0])
#print(dec_train[0])
print("Source Train:", enc_train.shape)
print("Target Train", dec_train.shape)

NameError: name 'tensor' is not defined

In [12]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, enc_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

NameError: name 'enc_train' is not defined

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

NameError: name 'tokenizer' is not defined

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, 
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=20)

NameError: name 'lyricist' is not defined

In [15]:
def generate_text(model, tokenizer, init_sentence='<start>', max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index['<end>']
    
    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
            
    generated = ''
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + ""
        
    return generated

In [16]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

NameError: name 'lyricist' is not defined